<a href="https://colab.research.google.com/github/raymondwcs/learning_spacy/blob/main/spaCy_training_(command_line%2C_English).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

https://zachlim98.github.io/me/2021-03/spacy3-ner-tutorial

In [1]:
!pip install spacy==3.0.6
!python -m spacy download en_core_web_lg


2021-08-11 04:21:57.630444: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
     |████████████████████████████████| 778.8 MB 20 kB/s 
  Attempting uninstall: en-core-web-lg
    Found existing installation: en-core-web-lg 3.1.0
    Uninstalling en-core-web-lg-3.1.0:
      Successfully uninstalled en-core-web-lg-3.1.0
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_lg')
2021-08-11 04:23:02.654872: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Auto-filled config with all values
✔ Saved config
config.cfg
You can now add your data and train your pipeline:
python -m spacy train config.cfg --paths.train ./train.spacy --paths.dev ./dev.spacy


In [ ]:
# https://spacy.io/usage/training#config

!python -m spacy init fill-config base_config.cfg config.cfg

In [12]:
# TRAIN_DATA = [
#  ('how much does a F35 cost', {'entities': [(16, 19, 'aircraft')]}),
#  ('would it be possible to steal a F15', {'entities': [(32, 35, 'aircraft')]}),
#  ('who manufactures the F16', {'entities': [(21, 24, 'aircraft')]}),
#  ('how many countries have bought the F35',
#   {'entities': [(35, 38, 'aircraft')]}),
#  ('is the F35 a waste of money', {'entities': [(7, 10, 'aircraft')]})]

TRAIN_DATA = [('The F15 aircraft uses a lot of fuel', {'entities': [(4, 7, 'aircraft')]}),
 ('did you see the F16 landing?', {'entities': [(16, 19, 'aircraft')]}),
 ('how many missiles can a F35 carry', {'entities': [(24, 27, 'aircraft')]}),
 ('is the F15 outdated', {'entities': [(7, 10, 'aircraft')]}),
 ('does the US still train pilots to dog fight?',
  {'entities': [(0, 0, 'aircraft')]}),
 ('how long does it take to train a F16 pilot',
  {'entities': [(33, 36, 'aircraft')]}),
 ('how much does a F35 cost', {'entities': [(16, 19, 'aircraft')]}),
 ('would it be possible to steal a F15', {'entities': [(32, 35, 'aircraft')]}),
 ('who manufactures the F16', {'entities': [(21, 24, 'aircraft')]}),
 ('how many countries have bought the F35',
  {'entities': [(35, 38, 'aircraft')]}),
 ('is the F35 a waste of money', {'entities': [(7, 10, 'aircraft')]})]

import pandas as pd
from tqdm import tqdm
import spacy
from spacy.tokens import DocBin

nlp = spacy.blank("en") # load a new spacy model
db = DocBin() # create a DocBin object

for text, annot in tqdm(TRAIN_DATA): # data in previous format
    doc = nlp.make_doc(text) # create doc object from text
    ents = []
    for start, end, label in annot["entities"]: # add character indexes
        span = doc.char_span(start, end, label=label)
        if span is None:
            pass
        else:
            ents.append(span)
    doc.ents = ents # label the text with the ents
    db.add(doc)

db.to_disk("./train.spacy") # save the docbin object


100%|██████████| 5/5 [00:00<00:00, 787.78it/s]


In [16]:
!python -m spacy train config.cfg --output ./output --paths.train ./train.spacy --paths.dev ./train.spacy --gpu-id 0

2021-08-11 04:38:04.701741: I tensorflow/stream_executor/platform/default/dso_loader.cc:53] Successfully opened dynamic library libcudart.so.11.0
✔ Created output directory: output
ℹ Using GPU: 0

=========================== Initializing pipeline ===========================
[2021-08-11 04:38:06,686] [INFO] Set up nlp object from config
[2021-08-11 04:38:06,696] [INFO] Pipeline: ['tok2vec', 'ner']
[2021-08-11 04:38:06,701] [INFO] Created vocabulary
[2021-08-11 04:38:09,837] [INFO] Added vectors: en_core_web_lg
[2021-08-11 04:38:09,838] [INFO] Finished initializing nlp object
[2021-08-11 04:38:10,213] [INFO] Initialized pipeline components: ['tok2vec', 'ner']
✔ Initialized pipeline

============================= Training pipeline =============================
ℹ Pipeline: ['tok2vec', 'ner']
ℹ Initial learn rate: 0.001
E    #       LOSS TOK2VEC  LOSS NER  ENTS_F  ENTS_P  ENTS_R  SCORE 
---  ------  ------------  --------  ------  ------  ------  ------
  0       0          0.00     18.17  

In [18]:
nlp1 = spacy.load("./output/model-best") #load the best model
doc = nlp1("Did you see the F16 fly by just now?") # input sample text

spacy.displacy.render(doc, style="ent", jupyter=True) # display in Jupyter